In [1]:
import os
import rasterio as rio
import numpy as np
import geopandas as gpd
import pandas as pd
from osgeo import gdal
import matplotlib.pyplot as plt
from progressbar import ProgressBar

from utils import *

In [2]:
plt.rcParams["figure.figsize"] = (28,10)
plt.ion()

In [3]:
data_dir = 'data_100m'

dem_file            = f'{data_dir}/dem.tif'
slope_file          = f'{data_dir}/slope.tif'
aspect_file         = f'{data_dir}/aspect.tif'
northing_file       = f'{data_dir}/northing.tif'
easting_file        = f'{data_dir}/easting.tif'

urban_distance_file = f'{data_dir}/urban_distance.tiff'
roads_distance_file = f'{data_dir}/roads_distance.tiff'
crops_distance_file = f'{data_dir}/crops_distance.tiff'

vegetation_agg_file = f'{data_dir}/vegetation_agg.tiff'
vegetation_file     = f'{data_dir}/vegetation.tiff'

fires_shp           = 'shapefiles/perimetrazioni_1997_2017.shp'
vegetation_shp      = 'shapefiles/tipiforestali_usosuolo_ctr_fv.shp'
roads_shp      = 'shapefiles/Tratte_stradali.shp'

In [4]:
with rio.open(dem_file) as src:
    print(f'Reading dem file {dem_file}')
    dem = src.read(1)
    dem[dem <= -9999] = np.NaN
_, dx, _, _, _, dy = src.transform.to_gdal()    
bbox = src.bounds

Reading dem file data_100m/dem.tif


## PREPROCESSING

In [5]:
with rio.open(dem_file) as src:
    print(f'Reading dem file {dem_file}')
    dem = src.read(1)
    dem[dem <= -9999] = np.NaN

if not os.path.isfile(slope_file):
    print(f'Creating slope file {slope_file}')
    gdal.DEMProcessing(slope_file, dem_file, 'slope')

    
if not os.path.isfile(northing_file) or not os.path.isfile(easting_file):

    if not os.path.isfile(aspect_file):
        print(f'Creating aspect file {aspect_file}')
        gdal.DEMProcessing(aspect_file, dem_file, 'aspect')

    with rio.open(aspect_file) as f:
        print(f'Calculating northing and easting files')
        print(f'Reading aspect file {aspect_file}')
        aspect = f.read(1)
        aspect[aspect <= -9999] = np.NaN    
        northing = np.cos(aspect * np.pi/180.0)
        easting = np.sin(aspect * np.pi/180.0)

    print(f'Saving northing file {northing_file}')
    save_raster_as(northing, northing_file, aspect_file)
    print(f'Saving easting file {easting_file}')    
    save_raster_as(easting, easting_file, aspect_file)

Reading dem file data_100m/dem.tif


In [6]:
print(f'Reading vegetation shp {vegetation_shp}')
vegetation = gpd.read_file(vegetation_shp)
vegetation['encoded_decodifica'], mapping = encode_feature(vegetation, 'Decodifica')    

if not os.path.isfile(vegetation_agg_file):
    print(f'Rasterizing vegetation')
    vegetation_raster = rasterize_numerical_feature(vegetation, dem_file, 'encoded_decodifica')
    print(f'Writing vegetation_raster file {vegetation_agg_file}')    
    save_raster_as(vegetation_raster, vegetation_file, dem_file)
    

if not os.path.isfile(vegetation_agg_file):
    print(f'Rasterizing vegetation_agg')    
    vegetation_agg_raster = rasterize_numerical_feature(vegetation, dem_file, 'id_agg_fv')
    print(f'Writing vegetation_agg_raster file {vegetation_agg_file}')
    save_raster_as(vegetation_agg_raster, vegetation_agg_file, dem_file)
    
if not os.path.isfile(vegetation_agg_file):
    print(f'Rasterizing vegetation_agg')    
    vegetation_agg_raster = rasterize_numerical_feature(vegetation, dem_file, 'id_agg_fv')
    print(f'Writing vegetation_agg_raster file {vegetation_agg_file}')
    save_raster_as(vegetation_agg_raster, vegetation_agg_file, dem_file)
    

Reading vegetation shp shapefiles/tipiforestali_usosuolo_ctr_fv.shp


In [9]:
vegetation_agg_raster = read_tiff(vegetation_agg_file)
mask = vegetation_agg_raster >= 20
indices = np.argwhere(mask)
coordinates = extract_coordinates(indices, src)


Reading file data_100m/vegetation_agg.tiff


In [10]:
if not os.path.isfile(crops_distance_file):
    crops = vegetation.query(f'Coltivo == 1')
    crops_raster_file = f'{data_dir}/crops.tiff'
    crops_raster = rasterize_numerical_feature(crops, dem_file)
    save_raster_as(crops_raster, crops_raster_file, dem_file)

    write_distance_raster(crops_raster_file, crops_distance_file)

In [11]:
if not os.path.isfile(urban_distance_file):
    urban = vegetation.query(f'Urbano == 1')
    urban_raster_file = f'{data_dir}/urban.tiff'
    urban_raster = rasterize_numerical_feature(urban, dem_file)
    save_raster_as(urban_raster, urban_raster_file, dem_file)

    urban_distance_file = f'{data_dir}/urban_distance.tiff'            
    write_distance_raster(urban_raster_file, urban_distance_file)

In [12]:
if not os.path.isfile(roads_distance_file):
    print(f'Reading roads shp {roads_shp}')
    roads = gpd.read_file(roads_shp)

    roads_raster = rasterize_numerical_feature(roads, dem_file)
    roads_raster_file = f'{data_dir}/roads.tiff'
    save_raster_as(roads_raster, roads_raster_file, dem_file)

    roads_distance_file = f'{data_dir}/roads_distance.tiff'            
    write_distance_raster(roads_raster_file, roads_distance_file)

## DATA PREPARATION

In [ ]:
print(f'Reading northing file {slope_file}')
slope = read_tiff(slope_file)

print(f'Reading northing file {northing_file}')
northing = read_tiff(northing_file)
    
print(f'Reading easting file {easting_file}') 
easting = read_tiff(easting_file)

print(f'Reading vegetation_raster file {vegetation_file}')            
vegetation_raster = read_tiff(vegetation_file)
    
print(f'Reading vegetation_agg_raster file {vegetation_agg_file}')          
vegetation_agg_raster = read_tiff(vegetation_agg_file)

print(f'Reading urban distance file {urban_distance_file}')
urban_distance = read_tiff(urban_distance_file)

print(f'Reading roads distance file {roads_distance_file}')
roads_distance = read_tiff(roads_distance_file)

print(f'Reading roads distance file {crops_distance_file}')
crops_distance = read_tiff(crops_distance_file)

In [ ]:
print(f'Reading vegetation shp {vegetation_shp}')
vegetation = gpd.read_file(vegetation_shp)
vegetation['encoded_decodifica'], mapping = encode_feature(vegetation, 'Decodifica')

In [ ]:
# masking vegetation raster excluding non-vegetated areas
mask = (vegetation_agg_raster >= 20) & (dem != -9999) & (slope != -9999) & (northing != -9999) & (easting != -9999)
indeces = np.argwhere(mask)
coordinates = extract_coordinates(indeces, src)
points_geom = [Point(*p) for p in coordinates.T]

points = gpd.GeoDataFrame(pd.DataFrame(indeces, columns=['row', 'col']), geometry=points_geom, crs={'init': 'epsg:3003'})

points_envelopes = points.copy()
points_envelopes.geometry = points.geometry.buffer(dx/2).envelope

In [ ]:
print(f'Reading fires shp {fires_shp}')
fires = gpd.read_file(fires_shp)

presence_df = gpd.sjoin(points_envelopes, fires)\
             .loc[:, ('row', 'col', 'data', 'anno', 'stagione', 'area_ha')]

In [ ]:
absence_idx = ~points.index.isin(presence_df.index)
absence_df = points.iloc[points.index[absence_idx]]

presence_df.loc[:, 'fire'] = 1
absence_df.loc[:, 'fire'] = 0

dataset = pd.concat((presence_df, absence_df), axis=0)

In [ ]:
data_dict = {
    'dem': dem,
    'slope': slope,
    'north': northing,
    'east': easting,
    'veg': vegetation_raster,
    'veg_agg': vegetation_agg_raster,
    'urban_d': urban_distance,
    'roads_d': roads_distance,
    'crops_d': crops_distance
}

for k, v in data_dict.items():
    dataset[k] = v[dataset.row.values, dataset.col.values]

In [ ]:
dataset.loc[dataset['north'].isna(), 'north'] = 0
dataset.loc[dataset['east'].isna(), 'east'] = 0
dataset.loc[dataset['roads_d'].isna(), 'roads_d'] = -9999   

dataset.geometry = points.loc[dataset.index].geometry

dataset_csv = dataset.copy()
dataset_csv['x'] = dataset_csv.geometry.x
dataset_csv['y'] = dataset_csv.geometry.y
dataset_csv = dataset_csv.drop('geometry', axis=1)

dataset_csv.to_csv(f'{data_dir}/dataset.csv')

dataset.to_file(f'{data_dir}/dataset.geojson', driver='GeoJSON')